In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
Renewable_Energy = "Solar_PBE"

try:
    address = "/content/drive/My Drive/Doctor/Research/DRL/Colab/"
    data_train_csv1 = pd.read_csv(address+Renewable_Energy+'_16.csv', index_col=0)

except:
    address = ""
    data_train_csv1 = pd.read_csv(address+Renewable_Energy+'_16.csv', index_col=0)

data_train_csv2 = pd.read_csv(address+Renewable_Energy+'_17.csv', index_col=0)
data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
data_val_csv    = pd.read_csv(address+Renewable_Energy+'_18.csv', index_col=0)
data_test_csv   = pd.read_csv(address+Renewable_Energy+'_19.csv', index_col=0)

In [ ]:
# Data Preprocessing
 
Battery_Size = 0.5
unit         = 4 #unit: 15 minute
 
RE_Capacity1 = max(data_train_csv['Power(MW)'])
RE_Capacity2 = max(data_val_csv['Power(MW)'])
RE_Capacity3 = max(data_test_csv['Power(MW)'])
 
size_train0 = int(len(data_train_csv)/unit)
size_val0   = int(len(data_val_csv)/unit)
size_test0  = int(len(data_test_csv)/unit)
 
data_train0 = []
data_train  = []
for i in range(size_train0):
    data_train0 += [round(pd.Series.mean(data_train_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity1, 3)]
    data_train  += [data_train0[i]] if data_train0[i] > 0 else []
 
data_val0 = []
data_val  = []
for i in range(size_val0):
    data_val0 += [round(pd.Series.mean(data_val_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity2, 3)]
    data_val  += [data_val0[i]] if data_val0[i] > 0 else []
 
data_test0 = []
data_test  = []
for i in range(size_test0):
    data_test0 += [round(pd.Series.mean(data_test_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity3, 3)]
    data_test  += [data_test0[i]] if data_test0[i] > 0 else []

In [ ]:
# PPO Agent (Partailly Observable State, Continuous Action Space)
# Assumption 1: Standard deviation is fixed
# Assumption 2: History is composed of observations only

n_layers         = 1
in_size          = 2
hidden_size      = 16
out_size         = 1
T_horizon        = 128
learning_rate    = 0.001
K_epoch          = 3
gamma            = 0.99
lmbda            = 0.95
eps_clip         = 0.01
C_value          = 1
var              = 0.1**2

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc_s  = nn.Linear(in_size, hidden_size)
        self.rnn   = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc_pi = nn.Linear(hidden_size, out_size)
        self.fc_v  = nn.Linear(hidden_size, 1)

    def pi(self, x, hidden):
        x = F.relu(self.fc_s(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        pi = self.fc_pi(x)
        pi = F.relu(pi.view(-1, out_size))
        return pi, hidden
    
    def v(self, x, hidden):
        x = F.relu(self.fc_s(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        v = self.fc_v(x)
        v = v.view(-1, 1)
        return v

def train_net(model, batch, optimizer):
    o, H, a, r, o_prime, H_prime, done = [], [], [], [], [], [], []
    for transition in batch:
        o.append(transition[0])
        H.append(transition[1])
        a.append(transition[2])
        r.append([transition[3]])
        o_prime.append(transition[4])
        H_prime.append(transition[5])
        done.append([0]) if transition[6] else done.append([1])
        
    o         = torch.tensor(o,dtype=torch.float)
    H         = (H[0][0].detach(), H[0][1].detach())
    a         = torch.tensor(a,dtype=torch.float)
    r         = torch.tensor(r,dtype=torch.float)
    o_prime   = torch.tensor(o_prime,dtype=torch.float)
    H_prime   = (H_prime[0][0].detach(), H_prime[0][1].detach())
    done      = torch.tensor(done)

    pdf_old = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
    prob_old = torch.exp(pdf_old.log_prob(a)).view(len(a),1)
    prob_old = prob_old.detach()

    v_target = r + gamma * model.v(o_prime, H_prime) * done
    td = r + gamma * model.v(o_prime, H_prime) * done - model.v(o, H)
    td = td.detach().numpy()
    advantage = []
    A = 0.0
    for delta in td[::-1].flatten():
        A = delta + gamma*lmbda*A
        advantage.append([A])
    advantage.reverse()
    advantage = torch.tensor(advantage, dtype=torch.float)
    
    for i in range(K_epoch):
        pdf = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
        prob = torch.exp(pdf.log_prob(a)).view(len(a),1)
        ratio = torch.exp(torch.log(prob) - torch.log(prob_old))  # a/b == exp(log(a)-log(b))

        loss_actor = torch.min(ratio * advantage, torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage)
        loss_critic = F.mse_loss(model.v(o, H), v_target.detach())
        loss = -(loss_actor - C_value*loss_critic)
        
        optimizer.zero_grad()
        loss.mean().backward(retain_graph=True)
        optimizer.step()

In [ ]:
# Environment

E_max   = Battery_Size
tdelta  = unit/4
eff_c   = 0.9
eff_d   = 0.9
soc_min = 0.1
soc_max = 0.9
P_cmax  = Battery_Size/3
P_dmax  = Battery_Size/3
beta_c  = 0.01
beta_d  = 0.01

class Env():
    def __init__(self, data):
        self.data = data
        self.state = []

    def reset(self):
        real = self.data[0]
        E = E_max/2
        state = [[real, E]]
        self.state = state
        return state

    def step(self, action):
        real = self.data[len(state)]
        pred = max(action[0], 0.0)

        E = state[-1][1]
        P_climit = min(P_cmax, (1/eff_c)*(E_max*soc_max - E)/tdelta)
        P_dlimit = min(P_dmax, eff_d*(E - E_max*soc_min)/tdelta)
        P_c = min(max(real-pred, 0), P_climit)
        P_d = min(max(pred-real, 0), P_dlimit)
        E_prime = E + eff_c*P_c*tdelta - (1/eff_d)*P_d*tdelta
        disp = real - P_c + P_d
        error = pred - disp
        error_function = abs(error) + beta_c*P_c + beta_d*P_d

        next_state = state + [[real, E_prime]]
        reward = -error_function
        done = False
        info = [real, pred, disp]

        self.state = next_state
        return next_state, reward, done, info

In [ ]:
# PPO Training
 
total_episode = 50
max_iteration = int(len(data_train)/T_horizon)
print_interval = 1
 
model = LSTM()
env_train = Env(data_train)
env_val   = Env(data_val)
env_test  = Env(data_test)
pred_train, pred_val, pred_test = [], [], [] # Predicted Value
mape_train, mape_val, mape_test = [], [], [] # Mean Absolute Percentage Error
ccr_train, ccr_val, ccr_test    = [], [], [] # Complete Compensation Ratio

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for n_epi in range(total_episode):
    pred_train += [[]]; pred_val += [[]]; pred_test += [[]]
    mape_train += [[]]; mape_val += [[]]; mape_test += [[]]
    ccr_train  += [[]]; ccr_val  += [[]]; ccr_test  += [[]]
 
    state = env_train.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for i in range(max_iteration):
        batch = []
        for t in range(T_horizon):
            pi_out, next_history = model.pi(torch.tensor(state[-1], dtype=torch.float), history)
            action = np.random.multivariate_normal(pi_out.detach().numpy()[0], var*np.identity(out_size), 1)[0].tolist()
            next_state, reward, done, info = env_train.step(action)
 
            batch.append((state[-1], history, action, reward, next_state[-1], next_history, done))
            state = next_state[:]
            history = next_history
 
            real = info[0]; pred = info[1]; disp = info[2]
            pred_train[n_epi]  += [pred]
            mape_train[n_epi]  += [abs((pred-disp)/disp)] if disp != 0 else [0]
            ccr_train[n_epi] += [1 if pred == disp else 0]
            if done:
                break
        
        if n_epi != 0:
            train_net(model, batch, optimizer)
        if done:
            break
    
    state = env_val.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for k in range(len(env_val.data)-1):
        pi_out, next_history = model.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_val.step(action)
 
        state = next_state[:]
        history = next_history
        
        real = info[0]; pred = info[1]; disp = info[2]
        pred_val[n_epi]  += [pred]
        mape_val[n_epi]  += [abs((pred-disp)/disp)] if disp != 0 else [0]
        ccr_val[n_epi] += [1 if pred == disp else 0]
    
    state = env_test.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for l in range(len(env_test.data)-1):
        pi_out, next_history = model.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_test.step(action)
 
        state = next_state[:]
        history = next_history
        
        real = info[0]; pred = info[1]; disp = info[2]
        pred_test[n_epi]  += [pred]
        mape_test[n_epi]  += [abs((pred-disp)/disp)] if disp != 0 else [0]
        ccr_test[n_epi] += [1 if pred == disp else 0]
    
    if (n_epi+1)%print_interval == 0:
        MAPE_train = round(100*np.mean(mape_train[n_epi]),2)
        MAPE_val   = round(100*np.mean(mape_val[n_epi]),2)
        MAPE_test  = round(100*np.mean(mape_test[n_epi]),2)
        CCR_train  = round(np.mean(ccr_train[n_epi]),3)
        CCR_val    = round(np.mean(ccr_val[n_epi]),3)
        CCR_test   = round(np.mean(ccr_test[n_epi]),3)
 
        print("episode: {}".format(n_epi+1))
        print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
        print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
        print("MAPE_test: {}%".format(MAPE_test).ljust(25))
        print("CCR_train: {}".format(CCR_train).ljust(25), end="")
        print("CCR_val: {}".format(CCR_val).ljust(25), end="")
        print("CCR_test: {}".format(CCR_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

episode: 1
MAPE_train: 432.5%       MAPE_val: 389.77%        MAPE_test: 381.58%       
CCR_train: 0.252         CCR_val: 0.209           CCR_test: 0.219          
------------------------------------------------------------------------------------------
episode: 2
MAPE_train: 417.68%      MAPE_val: 309.8%         MAPE_test: 297.71%       
CCR_train: 0.256         CCR_val: 0.265           CCR_test: 0.277          
------------------------------------------------------------------------------------------
episode: 3
MAPE_train: 204.68%      MAPE_val: 153.78%        MAPE_test: 147.82%       
CCR_train: 0.312         CCR_val: 0.37            CCR_test: 0.382          
------------------------------------------------------------------------------------------
episode: 4
MAPE_train: 148.21%      MAPE_val: 142.98%        MAPE_test: 135.23%       
CCR_train: 0.325         CCR_val: 0.37            CCR_test: 0.387          
---------------------------------------------------------------------------

In [ ]:
# Produce results
#
#select_num = np.argmin(np.mean(mape_val,axis=1))
#select = pd.DataFrame(np.array(pred_test[select_num][:]))
#select.to_csv(address+Renewable_Energy+"_Model3_DeepComp_"+str(int(100*E_max))+".csv")